In [1]:
#!/usr/bin/env perl
use strict;
use warnings;
use feature qw/ say current_sub /;
use Data::Dumper;
use lib '.';
use toolkit;
use graphql;
use adventofcode;

$Data::Dumper::Sortkeys = 1;

1

In [3]:
my $practice_input = qq/
............
........0...
.....0......
.......0....
....0.......
......A.....
............
............
........A...
.........A..
............
............
/;


............
........0...
.....0......
.......0....
....0.......
......A.....
............
............
........A...
.........A..
............
............


In [104]:
sub group_by {
    my ($fun, @args) = @_;
    my %table;
    if (ref $fun eq 'CODE') {
        foreach (@args) {
            push @{$table{$fun->($_)}}, $_;
        }
    } else {
        foreach (@args) {
            push @{$table{$_->{$_}}}, $_;
        }
    }
    return %table;
}
sub map_groups {
    my ($fun, %groups) = @_;
    my %res = map $fun->($_, $groups{$_}), keys %groups;
    return %res;
}

sub deltap {
    my ($a, $b) = @_;
    return [ map { $b->[$_] - $a->[$_] } 0 .. $#$a ];
}

sub addp {
    my ($a, $b) = @_;
    return [ map { $b->[$_] + $a->[$_] } 0 .. $#$a ];
}
sub in_bounds {
    my ($p, @shape) = @_;
    return $p->[0] >= 0
        && $p->[1] >= 0
        && $p->[0] < $shape[0]
        && $p->[1] < $shape[1];
}

sub process_day8_part1 {
    my ($input) = @_;
    my @shape = shape(parse_2d_map_array($input));
    
    my %letter_map =
        map_groups sub { $_[0] => [ map $_->{coord}, @{$_[1]} ] },
        group_by selector('value'),
        flatten_nd
        map_nd_indexed graphql_query('{ value coord }'),
        parse_2d_map_array($input);
    
    my %antinodes =
        map { $_->[0].','.$_->[1] => $_ }
        map @{$_->{antinodes}},
        grep defined,
        flatten_nd
        map_nd sub {
            my $self = $_;
            return undef if $self->{value} eq '.';
            return {
                %$self,
                antinodes => [
                    grep { in_bounds($_, @shape) }
                    map { addp($_, addp($_, $self->{coord})) }
                    grep { $_->[0] != 0 || $_->[1] != 0 }
                    map { deltap($self->{coord}, $_) }
                    @{$letter_map{$self->{value}}}
                ]
            }
        },
        map_nd_indexed graphql_query('{ value coord }'),
        parse_2d_map_array($input);

    my $transformed_map = string_2d_map_array
        map_nd sub { exists $antinodes{$_->{coord}[0].','.$_->{coord}[1]} ? '#' : $_->{value} },
        map_nd_indexed graphql_query('{ value coord }'),
        parse_2d_map_array($input);

    say "debug: transformed_map:\n$transformed_map";
    sum
        map 1,
        grep /#/,
        flatten_nd
        parse_2d_map_array($transformed_map)
    # @shape;
    # \%antinodes;
}

say Dumper process_day8_part1($practice_input);

debug: transformed_map:
......#....#
...#....0...
....#0....#.
..#....0....
....0....#..
.#....#.....
...#........
#......#....
........A...
.........A..
..........#.
..........#.
$VAR1 = 14;



1

Warning: Subroutine group_by redefined at reply input line 1.

Subroutine map_groups redefined at reply input line 15.

Subroutine deltap redefined at reply input line 21.

Subroutine addp redefined at reply input line 26.

Subroutine in_bounds redefined at reply input line 30.

Subroutine process_day8_part1 redefined at reply input line 38.


In [105]:
my $input = get_challenge('2024/day/8/input');
my $res = process_day8_part1($input);

debug: transformed_map:
........................#.#......#..8...........##
..................E...#........#...........#..#.#.
#................................g.#.....#........
..#.......#......#..............###.......#l#.#b..
..C#..##......s...##.........8......#...b.#.#..#..
...#..............3.#1...........##.......#...#...
....##......N....3...#..#..............1.....b....
........#.......#....N.##..8....#...........##.2..
..q.....#.............#..#..#.#.#..#...P..#.#.....
.............#......#.N..#..#..............#......
...#......#E.......#.....#...........#.......l..#.
........#.##.S.....c...#.........T..#v.....#......
.....#...w....E........q........#...L.....P.....l.
....#...w.....................##.......a...#......
.......#.#.t.....##...........v..V........#...#...
.....w.C#..............#.#.#........V....4.....L..
...........................#...........#I.n.##....
..#..E.5..C...8....3.###.....#....#............#..
.#.#...#.....#.s..0...A........W...........a...#T.
....#..

280

In [106]:
say post_answer('2024/day/8/answer', 1, $res);

<!DOCTYPE html>
<html lang="en-us">
<head>
<meta charset="utf-8"/>
<title>Day 8 - Advent of Code 2024</title>
<link rel="stylesheet" type="text/css" href="/static/style.css?31"/>
<link rel="stylesheet alternate" type="text/css" href="/static/highcontrast.css?1" title="High Contrast"/>
<link rel="shortcut icon" href="/favicon.png"/>
<script>window.addEventListener('click', function(e,s,r){if(e.target.nodeName==='CODE'&&e.detail===3){s=window.getSelection();s.removeAllRanges();r=document.createRange();r.selectNodeContents(e.target);s.addRange(r);}});</script>
</head><!--




Oh, hello!  Funny seeing you here.

I appreciate your enthusiasm, but you aren't going to find much down here.
There certainly aren't clues to any of the puzzles.  The best surprises don't
even appear in the source until you unlock them for real.

Please be careful with automated requests; I'm not a massive company, and I can
only take so much traffic.  Please be considerate so that everyone gets to play.

If you're 

1

In [110]:

sub process_day8_part2 {
    my ($input) = @_;
    my @shape = shape(parse_2d_map_array($input));
    
    my %letter_map =
        map_groups sub { $_[0] => [ map $_->{coord}, @{$_[1]} ] },
        group_by selector('value'),
        flatten_nd
        map_nd_indexed graphql_query('{ value coord }'),
        parse_2d_map_array($input);
    
    my %antinodes =
        map { $_->[0].','.$_->[1] => $_ }
        map @{$_->{antinodes}},
        grep defined,
        flatten_nd
        map_nd sub {
            my $self = $_;
            return undef if $self->{value} eq '.';
            return {
                %$self,
                antinodes => [
                    grep { in_bounds($_, @shape) }
                    map {
                        my $delta = $_;
                        my $p = addp($delta, $self->{coord});
                        my @resonance_ps;
                        while (in_bounds($p, @shape)) {
                            push @resonance_ps, $p;
                            $p = addp($delta, $p);
                        }
                        @resonance_ps;
                    }
                    grep { $_->[0] != 0 || $_->[1] != 0 }
                    map { deltap($self->{coord}, $_) }
                    @{$letter_map{$self->{value}}}
                ]
            }
        },
        map_nd_indexed graphql_query('{ value coord }'),
        parse_2d_map_array($input);

    my $transformed_map = string_2d_map_array
        map_nd sub { exists $antinodes{$_->{coord}[0].','.$_->{coord}[1]} ? '#' : $_->{value} },
        map_nd_indexed graphql_query('{ value coord }'),
        parse_2d_map_array($input);

    say "debug: transformed_map:\n$transformed_map";
    sum
        map 1,
        grep /#/,
        flatten_nd
        parse_2d_map_array($transformed_map)
    # @shape;
    # \%antinodes;
}

say Dumper process_day8_part2($practice_input);

debug: transformed_map:
##....#....#
.#.#....#...
..#.##....#.
..##...#....
....#....#..
.#...##....#
...#..#.....
#....#.#....
..#.....#...
....#....#..
.#........#.
...#......##
$VAR1 = 34;



1

Warning: Subroutine process_day8_part2 redefined at reply input line 2.


In [111]:
my $input = get_challenge('2024/day/8/input');
my $res = process_day8_part2($input);

debug: transformed_map:
..........##....#....#.####..##..#..###....#.#.###
.........#..#.#...#.#.#.#.#....#..#.......##..#.##
#....##..#......#..#.#..##..#...##.#..#..#......##
#.##....#.#..#...#####.#..#.#..#####.#....###.###.
..##..##.....##.####...#..##.#..#...#...#.#.#.##..
...#..#.##......#.#.##...........##.#.....#..##..#
....##...##.#.##.#...#..#.#...#........####.##...#
........#...#.#.####.#.##..#.####...###..#.###.#.#
..#....##.#.##.###...##..#..###.#####..#.####...#.
.....#......###...#.#.#..#..#...#.#....######.....
...#......##.#.....####..#..##..#.#.##..###..#.##.
.#..#...#.####.....#.####.....#.##..##.##..##.#...
.#...#...#.#..##...#...#...##.#.#...########....#.
#...#...#.#...........###.#...##....####..##......
.......#.#.##.#..##....##.....#.##.###....#...#...
.#...####.............##.#.##..#...##.##.###...#..
.....#.##..#.#......###.#..#......#######.#.##...#
#.######.##.#.#.#.#######.#.##...##.....#.#....##.
.#.#.#.#.....#.##.##..##.......###...#.....#.#.##.
.####..

958

In [112]:
say post_answer('2024/day/8/answer', 2, $res);

<!DOCTYPE html>
<html lang="en-us">
<head>
<meta charset="utf-8"/>
<title>Day 8 - Advent of Code 2024</title>
<link rel="stylesheet" type="text/css" href="/static/style.css?31"/>
<link rel="stylesheet alternate" type="text/css" href="/static/highcontrast.css?1" title="High Contrast"/>
<link rel="shortcut icon" href="/favicon.png"/>
<script>window.addEventListener('click', function(e,s,r){if(e.target.nodeName==='CODE'&&e.detail===3){s=window.getSelection();s.removeAllRanges();r=document.createRange();r.selectNodeContents(e.target);s.addRange(r);}});</script>
</head><!--




Oh, hello!  Funny seeing you here.

I appreciate your enthusiasm, but you aren't going to find much down here.
There certainly aren't clues to any of the puzzles.  The best surprises don't
even appear in the source until you unlock them for real.

Please be careful with automated requests; I'm not a massive company, and I can
only take so much traffic.  Please be considerate so that everyone gets to play.

If you're 

1